In [ ]:
!pip install transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.2 MB/s eta 0:00:00


In [ ]:
documents_kr = [
    "한국의 수도는 서울입니다.",
    "경복궁은 서울에 위치하고 있습니다.",
    "경복궁은 한국의 전통 궁전입니다."
]

In [ ]:
import faiss
from transformers import BertTokenizer, BertModel, AutoModelForCausalLM, AutoTokenizer
import numpy as np

# 문서 임베딩 생성 함수
def get_embeddings(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

# BERT 모델과 토크나이저 로드 (한국어 모델)
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertModel.from_pretrained('klue/bert-base')

# 문서 임베딩 생성
doc_embeddings = get_embeddings(documents_kr, model, tokenizer)

# FAISS 인덱스 생성
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# 검색 함수
def search(query, top_k=1):
    query_embedding = get_embeddings([query], model, tokenizer)
    D, I = index.search(query_embedding, top_k)
    return [documents_kr[i] for i in I[0]]

In [ ]:
# GPT-2 모델과 토크나이저 로드 (한국어 모델)
gpt2_tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
gpt2_model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')

# 답변 생성 함수
def generate_answer(query, context):
    input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    inputs = gpt2_tokenizer.encode(input_text, return_tensors='pt')
    outputs = gpt2_model.generate(inputs, max_length=100, num_return_sequences=1)
    return gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
# 사용자 질문
query = "한국의 수도는 어디인가요?"

# 관련 문서 검색
retrieved_docs = search(query, top_k=1)
context = retrieved_docs[0]

# 답변 생성
answer = generate_answer(query, context)
print(f"Question: {query}")
print(f"Answer: {answer}")

Question: 한국의 수도는 어디인가요?
Answer: Context: 한국의 수도는 서울입니다.

Question: 한국의 수도는 어디인가요?

Answer: 한국의 수도는 어디인가요?
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울은 서울입니다.
A: 서울
